# La structure d'une application Python


:::{note} Les objectifs de cette partie
- Comprendre la différence entre module et package Python
- Définir l'arborescence de son application
- Comprendre ce que fait `import` et contrôler l'usage de son application
- Construire son premier package en utilisant `pyproject.toml`
:::

Dans cette partie, nous allons voir comment se structure une application Python en donnant un exemple d'arborescence. Celle-ci n'est pas figée mais est utilisée dans la plupart des projets en calcul scientifique (numpy, pandas, scikit-learn) et autres.

Une application Python est constituée
- de fichiers Python avec l'extension `.py` que l'on appelle des **modules**,
- de répertoires contenant des fichiers Python que l'on appelle des **packages**.

Les modules et les packages peuvent être utilisés dans l'interpréteur Python en utilisant la commande `import`. Lorsque l'on crée sa propre application, il est important de bien comprendre leur fonctionnement afin de définir les comportements que l'on souhaite pour l'utilisateur final lorsqu'il importe l'ensemble ou une partie du package.

## Un module

Afin d'illustrer l'utilisation d'un module en Python, nous allons écrire un calculateur qui sait uniquement faire une addition et une soustraction.

Voici le fichier `calculator_mod.py`.

In [ ]:
%%file calculator_mod.py
"""
Calculator module
"""

def add(a, b):
    """
    return a + b
    """
    return a + b

def sub(a, b):
    """
    return a - b
    """
    return a - b

#### Utilisation d'un module

Il existe différentes manières d'importer un module en utilisant le mot-clef `import`.

1. On peut importer un module via son nom.

In [ ]:
import calculator_mod
calculator_mod.add(1, 2)

2. On peut importer une partie d'un module.

In [ ]:
from calculator_mod import sub
sub(1, 2)

3. On peut importer un module en modifiant son nom d'appel.

In [ ]:
import calculator_mod as calc
calc.add(1, 2)

4. On peut importer l'ensemble du module.

In [ ]:
from calculator_mod import *
add(1, 2)

:::{attention}
On évitera par la suite ce type d'import qui est jugé dangereux.
:::

`import` définit explicitement certains attributs du module

- `__dict__` : dictionnaire utilisé par le module pour l'espace de noms des attributs
- `__name__` : nom du module
- `__file__` : fichier du module
- `__doc__` : documentation du module

In [ ]:
print('file', calculator_mod.__file__)
print('name', calculator_mod.__name__)
print('doc', calculator_mod.__doc__)

:::{attention}
Lors de l'exécution d'un programme, le module n'est importé qu'une seule fois.  
:::

#### Exécution d'un module

On peut ajouter à la fin d'un module le test suivant:

In [ ]:
if __name__ == '__main__':
    print(add(1, 2))

On peut à présent exécuter le module.

In [ ]:
%%file calculator_mod.py
"""
Calculator module
"""

def add(a, b):
    """
    return a + b
    """
    return a + b

def sub(a, b):
    """
    return a - b
    """
    return a - b

if __name__ == '__main__':
    print(add(1, 2))

In [ ]:
! python calculator_mod.py

:::{tip} Remarque
La partie du `if` n'est exécutée que si le module est le programme principal.
:::

## Un package

Comme dit en introduction, un package est un ensemble de modules Python. Prenons l'arborescence suivante

In [ ]:
! tree examples/simple_calculator/

On trouve ici un package appelé `calculator` et un sous-package appelé `operator` dans lequel se trouvent deux modules (`add.py` et `sub.py`).

### Le fichier `__init__.py` 
:::{important} Le fichier `__init__.py`

- Il est obligatoire pour que Python considère les répertoires comme contenant des packages ou modules.
- Il peut être vide.
- Il peut contenir du code d'initialisation.
- Il peut contenir des `import`.
- Il peut contenir la variable `__all__`.

:::

Prenons l'exemple du fichier `calculator/operator/__init__.py`

In [ ]:
__all__ = ['add', 'sub']

De cette manière, on peut importer `add` et `sub` en faisant tout simplement

In [ ]:
import sys
sys.path.append("./examples/simple_calculator/")

In [ ]:
from calculator.operator import *

On accède ensuite aux attributs et aux fonctions en faisant

In [ ]:
print(add.add(1, 2))
print(sub.sub(1, 2))

Il est également important d'utiliser l'import relatif lorsque l'on utilise des fonctionnalités de notre application dans les différents modules.   

Prenons l'example du fichier `calculator/__init__.py`

In [ ]:
from . import operator
from .operator import *
from .operator.add import add

On a alors le comportement suivant

In [ ]:
import calculator
calculator.add(1, 2)

In [ ]:
calculator.sub.sub(2,3)

:::{exercise} 
:label: import
Que peut-on faire d'autres ?

1. `calculator.add.add(1, 2)` ?
2. `calculator.sub.sub(1, 2)` ?
2. `calculator.operator.sub(1, 2)` ?
2. `calculator.operator.add.add(1, 2)` ?
:::

:::{solution} import
:class: dropdown

1. non
2. oui
3. non
4. oui
:::

## Recherche de modules et de packages

Pour que Python importe correctement un module, celui-ci doit être dans son **PATH**. Le module `sys` permet de connaître la liste des répertoires où Python va rechercher les modules.

In [ ]:
import sys
print(sys.path)

Python va donc rechercher dans

- le répertoire courant
- dans **PYTHONPATH** si défini (c'est la même syntaxe que le **PATH**)
- dans un répertoire par défaut

On peut également rajouter des répertoires à l'exécution étant donné que `sys.path` n'est qu'une liste.

In [ ]:
sys.path.append("/home/loic/Formations/")
print(sys.path)

Lorsque l'on veut importer **foo**, voici l'ordre des fichiers recherchés dans **sys.path**.

- **foo.dll**, **foo.dylib** ou **foo.so**
- **foo.py**
- **foo.pyc**
- **foo/\_\_init__.py**


:::{important}
L'ajout d'une nouvelle entrée dans la liste `sys.path` est montré uniquement pour comprendre le fonctionnement de l'import de package Python. Il ne faut en aucun cas utiliser cette méthode pour votre application. Nous allons voir dans la section suivante comment faire les choses proprement.
:::

## Réaliser sa première distribution

:::{caution} Attention
A partir du moment où l'on **diffuse**, il est indispensable de mettre une **licence**.
:::

Le contenu à diffuser peut être de différents types

- des modules et des sous packages
- des données
- des scripts
- des dépendances

:::{note}
Seul le premier item n'est pas optionnel. Vous pourriez ne pas indiquer les dépendances de votre application mais les conséquences seront que l'utilisateur pourra installer votre application mais il ne pourra pas l'utiliser.
:::

Il est nécessaire d'ajouter un ensemble de fichiers pour pouvoir faire le packaging.

Les indispensables

- `pyproject.toml`
- `README.rst` ou `README.md`
- `LICENSE.txt`
- votre `package`

Les optionnels

- `MANIFEST.in`

Voici une arborescence classique d'un package Python

    package/
        doc/
        examples/
        package/
            ...
            tests/
        tests/
        LICENSE.txt
        README.rst
        pyproject.toml

:::{note}
On pouvait également utiliser un fichier `setup.py` et `setup.cfg` pour définir notre package mais c'est à présent obsolète et remplacé par `pyproject.toml`.
:::

### Premier exemple de `pyproject.toml`

Ce fichier est l'élément central de votre application. Il va non seulement décrire la façon de construire le package, de le définir ainsi que ses dépendances à l'installation mais il va également être utilisé pour la configuration de tous les outils qui vont vérifier la qualité de l'application (black, ruff, pytest, ...).

```toml
[build-system]
requires = ["setuptools"]
build-backend = "setuptools.build_meta"

[project]
name = "calculator"
dynamic = ["version"]
requires-python = ">=3.8"

[tool.setuptools]
packages = ["calculator"]

[tool.setuptools.dynamic]
version = { attr = "calculator.version.__version__" }
```

- `build-system` indique les outils pour construire le package ainsi que le backend utilisé. Nous utiliserons tout au long de cet atelier `setuptools` mais il en existe bien d'autres[^buildsystem] (Hatch, PDM, Flit, Whey, ...).
  
[^buildsystem]: https://learn.scientific-python.org/development/guides/packaging-simple/

- `project` définit les caractéristiques de votre package (son nom, sa version, ses dépendances, sa description, ses auteurs, ...).

Pour installer votre package, il suffit de faire

1. `pip install .` pour installer le package dans le répertoire `site-packages` où se trouve votre version de Python.

2. `pip install . --user` pour l'installer dans votre `$HOME`.

3. `pip install -e .` pour installer votre application en mode développement. Chaque fois que vous faites une modification, le package est automatiquement mis à jour (ce n'est en fait qu'un lien symbolique vers votre package).

:::{note}
Si vous souhaitez désinstaller votre package, c'est toujours la même commande

```bash
pip uninstall nom_package
```
:::

### Gérer les dépendances

Les dépendances sont gérées à travers le fichier `pyproject.toml`. `pixi`est en mesure de mettre à jour ce fichier lorsque vous utilisez la ligne de commande.

Supposons par exemple que votre package a une dépendance à NumPy pour que celui-ci fonctionne. Il vous suffit de faire

```shell
pixi add numpy
```

et vous verrez dans le fichier `pyproject.toml`

```toml
[tool.pixi.dependencies]
numpy = ">=2.0.0,<2.1"
```

Vous pouvez également l'ajouter à la main de la manière suivante

```toml
[project]
...
dependencies = [
  "numpy",
]
```

De la même manière que ce qu'on a vu pour la définition des environnements de développement, il est possible de spécifier les versions que l'on souhaite.

```toml
[project]
...
dependencies = [
  "numpy>=1.0,<1.13",
]
```

Il est également possible d'ajouter des dépendances optionnelles comme par exemple

```toml
[project.optional-dependencies]
test = ["pytest"]
doc = ["sphinx", "nbsphinx"]
```

On peut ensuite sélectionner ces options lors de l'installation

```bash
pip install .[test]
pip install .[test,doc]
```

Mais on préférera passer par `pixi` en faisant l'équivalent

```bash
pixi add -f test pytest
pixi add -f doc sphinx nbsphinx
```

ce qui ajoutera les sections suivantes dans le fichier `pyproject.toml`

```toml
[tool.pixi.feature.test.dependencies]
pytest = ">=7.2.0,<7.3"

[tool.pixi.feature.doc.dependencies]
sphinx = ">=7.3.7,<7.4"
nbsphinx = ">=0.9.4,<0.10"
```

Il est ensuite possible de créer des environnements spécifiques à partir des features à l'aide de la commmande

```bash
pixi project environment add test -f test
pixi project environment add doc -f doc
```

Ce qui aura comme conséquence d'ajouter les lignes suivantes dans le fichier `pyproject.toml`

```toml
[tool.pixi.environments]
test = ["test"]
doc = ["doc"]
```

Pour utiliser l'environnement, il suffit de le spécifier via l'option `-e` comme par exemple

```bash
pixi run -e test pytest
```

Nous reverrons les environnements dans les prochains chapitres lorsque nous parlerons de la documentation et des tests.

### Ajout de scripts

En plus des sous packages et modules Python, votre application peut avoir des scripts que l'on peut vouloir exécuter en ligne de commande. Ce sont là encore des fichier Python. Ils sont spécifiés dans le fichier `pyproject.toml`.

```toml
[project.scripts]
calculator-script = "calculator.command_line:main"
```

où `calculator-script` est le nom de l'exécutable.

### Ajouter une extension

Un package ne contient pas forcément uniquement des fichiers Python. On peut par exemple avoir des extensions écrites en `cython`. Ce n'est pas si simple de les prendre en compte avec le fichier `pyproject.toml`. Si vous souhaitez faire quelque chose de propre, il faut regarder du coté de NumPy ou de pandas. Ils utilise tous les deux le backend `meson` pour construire leurs extensions Cython.

Afin de ne pas complexifier l'atelier, nous utiliserons l'ancienne méthode qui se conjugue avec le fichier `pyproject.toml`. Pour ce faire, nous allons créer le fichier `setup.py` avec le contenu suivant

In [ ]:
from setuptools import setup, find_packages
from setuptools.extension import Extension
from Cython.Build import cythonize

extension = [Extension(name = "calculator.cython_mod",
                       sources = ["calculator/cython_mod.pyx"])
            ]

setup(
    ...
    ext_modules=cythonize(extension),
    ...
)

Il faut également ajouter `cython` dans les dépendances du `build-system`.

```toml
[build-system]
requires = ["setuptools", "cython"]
build-backend = "setuptools.build_meta"
```

:::{note} Remarque

Vous êtes maintenant en mesure d'installer localement votre application. Vous souhaitez maintenant la diffuser plus largement. Il vous faut renseigner un peu plus d'informations avant de pouvoir le faire: les `classifiers`.

```toml
[project]
...
classifiers = [
    "Development Status :: 3 - Alpha",
    "Intended Audience :: Science/Research",
    "Intended Audience :: Developers",
    "License :: OSI Approved :: BSD License",
    "Programming Language :: Cython",
    'Programming Language :: Python :: 3',
    'Programming Language :: Python :: 3.8',
    'Programming Language :: Python :: 3.9',
    'Programming Language :: Python :: 3.10',
    'Programming Language :: Python :: 3.11',
    'Programming Language :: Python :: 3.12',
    'Programming Language :: Python :: 3 :: Only',
    "Topic :: Software Development",
    "Topic :: Scientific/Engineering",
    "Operating System :: Microsoft :: Windows",
    "Operating System :: POSIX",
    "Operating System :: Unix",
    "Operating System :: MacOS"
]
```

:::

## Exercices

### Etape 1

Dans le répertoire `step0`, vous avez l'arborescence suivante

In [ ]:
! tree TPs/1.packaging/step0/


:::{exercise}
Remplissez le fichier `pyproject.toml` pour pouvoir installer de manière minimale l'application `splinart`. Vous ajouterez au dur et à mesure les dépendances nécessaires à l'aide de la commande `pixi`.
:::

:::{exercise}
Remplissez les fichiers `__init__.py` pour que les démos fonctionnent.
:::

### Etape 2

:::{exercise}
Ajoutez le fichier script dans le fichier `pyptoject.toml` et testez.
:::

### Etape 3

:::{exercise}
Ajoutez les informations nécessaires pour diffuser l'application

- l'auteur,
- la description,
- la licence,
- la version,
- les classifiers,
:::